In [ ]:
import numpy as np
import pandas as pd

complaints = pd.read_csv('data/cleaned/nyc-311-complaints.csv')
nyc_population = pd.read_csv('../data/cleaned/population-by-zip-nyc-2010.csv')

## What are the 10 most common complaint types in NYC in 2017?

In [ ]:
most_common = (
    complaints.groupby('complaint_type')
              .apply(lambda x: len(x))
              .nlargest(n=10)
              .to_frame(name='n_complaints')
              .reset_index()
)
most_common

## For each borough, how many of each of those 10 types were there in 2017?

In [ ]:
by_borough = (
    complaints.groupby(['borough', 'complaint_type'])
              .size()
              .rename('n_complaints')
              .reset_index()
)

filt = by_borough['complaint_type'].isin(most_common['complaint_type'])
by_borough.loc[filt, :]

## Considering all complaint types, calculate the complaints per person that gives an indication of which boroughs make the most complaints (i.e., the "biggest complainers").

In [ ]:
import pandas as pd
nyc_population = pd.read_csv('../data/cleaned/population-by-zip-nyc-2010.csv')

population_by_borough = (
    nyc_population.groupby('borough', as_index=False)
                  .population
                  .sum()
)
population_by_borough

In [ ]:
complaints_per_borough = (
    complaints.groupby('borough')
              .apply(lambda x: len(x))
              .to_frame('n_complaints')
              .reset_index()
              .merge(borough_population, on='borough')
              .assign(complaints_per_person=lambda x: x.n_complaints / x.population)
              .reset_index()
)
complaints_per_borough